In [1]:
import glob
import os
from osgeo import gdal
import numpy as np
import pandas as pd
import pylab as plt
import plotly.graph_objects as go
import h5py
from matplotlib.pyplot import *
import imageio
import rasterio
import plotly.express as px

In [2]:
# Glob together all of the Snow Fraction datasets.
snow_ds = glob.glob('*.h5')
snow_ds

['Sierra2001.h5',
 'Sierra2002.h5',
 'Sierra2003.h5',
 'Sierra2004.h5',
 'Sierra2005.h5',
 'Sierra2006.h5',
 'Sierra2007.h5',
 'Sierra2008.h5',
 'Sierra2009.h5',
 'Sierra2010.h5',
 'Sierra2011.h5',
 'Sierra2012.h5',
 'Sierra2013.h5',
 'Sierra2014.h5',
 'Sierra2015.h5',
 'Sierra2016.h5',
 'Sierra2017.h5',
 'Sierra2018.h5',
 'Sierra2019.h5']

In [3]:
# Get subdatasets of first snow fraction dataset ('Sierra2001.h5').
datasets = gdal.Open(snow_ds[0], gdal.GA_ReadOnly).GetSubDatasets()

#(sds[3] is to choose the 4th dataset in the subdirectory (i.e., snow fraction). 
#The second bracket [0] is needed to open the dataset.
snow_data = gdal.Open(datasets[3][0])

#Changes the selected dataset into an array.
snow_data_array = snow_data.ReadAsArray()

#Converts the variables to 'float' to allow us to convert NA values (255) to nans
#We also convert 0s to nans so that when plotted on base map, only areas where data is present are shown
snow_data_float=snow_data_array.astype('float')
snow_data_float[snow_data_float == 255] = np.nan

sf_test = np.transpose(snow_data_float)

## Calcuate the Monthly Average Snow Cover for Each Dataset

In [4]:
# Test to see if we can create a range of dates starting at '2000-10-01', the first date of our first year. (Water years count from the year prior).
month_date = pd.Series(pd.date_range("2000-10-01", periods = 365, freq="d"))

In [5]:
month_list = []
# Now that we know we can create a range of dates, we need need to be able to subset the dates by each month. 
# Let's first try with month 6. 
month = month_date[month_date.dt.month == 6]
# Subsets our test file by range of index values for month 6  (day 243 to day 272)
month_len = sf_test[:,:,(list(month.index))]
# Takes the mean of each cell in x and y dimensions over the specific month. 
# Axis 2 alligns with our 3rd dimension, which are days, in this case.  
mean = np.mean(month_len, axis = 2)
# Appends the values to an empty list.
month_list.append(mean)
# Converts list to an array. 
month_array = np.array(month_list)
# Our array has 3-dimensions (first dimension being month_mean) so we need to subset the data to be 2 dimensions. 
month_array = month_array[0,:,:]

In [6]:
# figure(figsize=(8, 8))
# plt.imshow(month_array,interpolation = 'nearest')

## For Loop for Calculating Monthly Average Snow Cover for each Dataset

In [7]:
time_list = []
for i in range(len(snow_ds)):
    # Get subdatasets of first snow fraction dataset ('Sierra2001.h5').
    datasets = gdal.Open(snow_ds[i], gdal.GA_ReadOnly).GetSubDatasets()

    #(sds[3] is to choose the 4th dataset in the subdirectory (i.e., snow fraction). 
    #The second bracket [0] is needed to open the dataset.
    snow_data = gdal.Open(datasets[3][0])

    #Changes the selected dataset into an array.
    snow_data_array = snow_data.ReadAsArray()

    #Converts the variables to 'float' to allow us to convert NA values (255) to nans
    #We also convert 0s to nans so that when plotted on base map, only areas where data is present are shown
    snow_data_float=snow_data_array.astype('float')
    snow_data_float[snow_data_float == 255] = np.nan
    snow_data_float[snow_data_float == 0 ] = np.nan

    sf_test = np.transpose(snow_data_float)
    
    # Make a variable for the starting year of each water year.
    year =  i + 2000
    # Creates a variable for the first date in the water year. 
    start_date = pd.to_datetime(str(year) + '-10-01')
    end_date = pd.to_datetime(str(int(year) + 1) + '-09-30')
    # Creates a list of datetimes based on each year in our dataset.
    year_month_date = pd.Series(pd.date_range(start = start_date, end = end_date, freq="d"))
    #Need to create an empty list to append our mean values to. 
    new_list = []
    # For loop to calculate the mean for each month per year. 
    for j in range (1, 13):
        # Subset the date year based on month.
        month = year_month_date[year_month_date.dt.month == j]
        # Subset dataset by each month.
        month_len = sf_test[:,:,(list(month.index))]
        # Take the mean of each month per year. 
        mean = np.mean(month_len, axis = 2)
        # Append mean values to list per year. 
        new_list.append(mean)
    # Append year lists to empty list. 
    time_list.append(new_list)
# Converts list to array. 
date_array = np.array(time_list)
np.shape(date_array)

(19, 12, 1841, 1334)

## Calculate Annual Mean Snow Cover for Each Year

In [8]:
year_list = []
for i in range(len(snow_ds)):
    # Get subdatasets of first snow fraction dataset ('Sierra2001.h5').
    datasets = gdal.Open(snow_ds[i], gdal.GA_ReadOnly).GetSubDatasets()

    #(sds[3] is to choose the 4th dataset in the subdirectory (i.e., snow fraction). 
    #The second bracket [0] is needed to open the dataset.
    snow_data = gdal.Open(datasets[3][0])

    #Changes the selected dataset into an array.
    snow_data_array = snow_data.ReadAsArray()

    #Converts the variables to 'float' to allow us to convert NA values (255) to nans
    #We also convert 0s to nans so that when plotted on base map, only areas where data is present are shown
    snow_data_float=snow_data_array.astype('float')
    snow_data_float[snow_data_float == 255] = np.nan
    snow_data_float[snow_data_float == 0 ] = np.nan

    sf_test = np.transpose(snow_data_float)
    annual_snow = np.nanmean(sf_test, axis = 2)
    year_list.append(annual_snow)
year_array = np.array(year_list)

C:\Users\rmunn\AppData\Local\Temp/ipykernel_21428/3483513727.py:20: RuntimeWarning: Mean of empty slice
  annual_snow = np.nanmean(sf_test, axis = 2)


## Calculate Anomolies Per Month Per Year

In [9]:
# Takes the mean of each monthly mean
# Note: to get true mean (weighted mean), we would need to take the mean of the cumulatice days of a single month, then divide by the number of days. 
# However, since the sample size only varies by one day every four years (leap years), this will give us near identical values. 
month_mean = np.mean(date_array, axis = 0)

# Create empty list to put values in 
monthly_anom = []
# Select year
for i in range(len(date_array[:])):
    year_anom = date_array[i]
    sub_list = []
    for j in range(len(year_anom[:])):
        month_anom = year_anom[j] - month_mean[j]
        sub_list.append(month_anom)
    monthly_anom.append(sub_list)
anom_array = np.array(monthly_anom)

In [11]:
np.shape(month_mean)

(12, 1841, 1334)

## Select Indivdual Years and Months

In [12]:
# Shape of list is year, month, ydim, xdim
np.shape(date_array)

# Select the first year of our dataset.
year_one = date_array[0,:,:,:]
# Transpose data so that the dataset is ordered as: [xdim, ydim, month]
year_one_transposed = np.transpose(year_one, (1,2,0))

#Select the first month of our first year
year_one_month_one = year_one_transposed[:,:,0]

#Select the first year of the anomaly dataset. 
year_one_anom = anom_array[0,:,:,:]

year_one_month_one_anom = year_one_anom[:,:,0]

In [13]:
np.shape(year_one_month_one_anom)

(12, 1841)

In [14]:
np.shape(year_one_month_one_anom)

(12, 1841)

## Convert Monthly Averages to Geotiffs

In [15]:
# x dimension of array
xdim = snow_data_array.shape[1]
# y dimension of array
ydim = snow_data_array.shape[2]
# Projection data of sample GeoTiff
projection = 'PROJCS["Albers Conical Equal Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PARAMETER["standard_parallel_2",40.5],PARAMETER["false_easting",0],PARAMETER["false_northing",-4000000],UNIT["meters",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'
# transformation data of array
# Pull refrencing matrix from h5 file.
ref_matrix_meta = snow_data.GetMetadata()['Grid_MODIS_GRID_500m_ReferencingMatrix'].split()
referencing_matrix = [int(ref_matrix_meta[2]), int(ref_matrix_meta[1]), int(ref_matrix_meta[0]), int(ref_matrix_meta[5]), int(ref_matrix_meta[4]), int(ref_matrix_meta[3])]

### Convert Single Layer to Geotiff

In [16]:
def SingleGeotiff(raster_name, data, height, width, geotransform, wkt):
    
    driver = gdal.GetDriverByName('GTiff')

    dataset = driver.Create(
        raster_name,
        width,
        height,
        1,
        gdal.GDT_Float32)

    dataset.SetGeoTransform((
     geotransform))

    dataset.SetProjection(wkt)
    dataset.GetRasterBand(1).WriteArray(data)
    dataset.FlushCache()  # Write to disk.
    return dataset, dataset.GetRasterBand(1) 

In [17]:
SingleGeotiff('year_one_month_one_anom.tif', year_one_month_one_anom, ydim, xdim, referencing_matrix, projection)

ValueError: array larger than output file, or offset off edge

## Convert Stacked H5 File into N Single Geotiffs

In [ ]:
year_one_t = np.transpose(year_one)
path = 'tif/2001'
for i in range(len(year_one) -1):
    dest = ('month' + str(i + 1) + 'year_2001.tif') 
    name = os.path.join(path, dest)
    data = year_one_transposed[:,:,i]
    SingleGeotiff(name, data, ydim, xdim, referencing_matrix, projection)

In [ ]:
wgs = glob.glob('tif/2001/month*year*.tif')
wgs

['tif/2001\\month10year_2001.tif',
 'tif/2001\\month11year_2001.tif',
 'tif/2001\\month1year_2001.tif',
 'tif/2001\\month2year_2001.tif',
 'tif/2001\\month3year_2001.tif',
 'tif/2001\\month4year_2001.tif',
 'tif/2001\\month5year_2001.tif',
 'tif/2001\\month6year_2001.tif',
 'tif/2001\\month7year_2001.tif',
 'tif/2001\\month8year_2001.tif',
 'tif/2001\\month9year_2001.tif']

In [ ]:
# Change the following variables to the file you want to convert (inputfile)
# and what you want to name your output file (outputfile)
for i in range(len(wgs)):
    inputfile = wgs[i]
    outputfile = "wgs_test" + str(i) + ".tif"
    #Do not change the following line, it will reproject the geotiff file
    ds = gdal.Warp(outputfile, inputfile, dstSRS="+proj=longlat +datum=WGS84 +no_defs", dstNodata = 0)

In [ ]:
range(len(year_one))

range(0, 12)

In [ ]:
np.shape(year_one_transposed)

(1841, 1334, 12)

In [ ]:
path = 'tif/2001'
dest = 'sample'
name = os.path.join(path, dest)
name

### Convert Stacked Array to Stacked Geotiff

In [ ]:
def StackedGeotiff(name, array, geo_transform, projection):
    
    driver = gdal.GetDriverByName('GTiff')

    DataSet = driver.Create(name, array.shape[2], array.shape[1], array.shape[0], gdal.GDT_Float32)
    DataSet.SetGeoTransform(geo_transform)
    DataSet.SetProjection(projection)
    for i, image in enumerate(array, 1):
        DataSet.GetRasterBand(i).WriteArray( image )
    DataSet.FlushCache()
    return name

In [ ]:
StackedGeotiff('year_one_stack.tif', year_one, referencing_matrix, projection)

'year_one_stack.tif'